In [5]:
import tensorflow as tf
import os
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import numpy as np
from pathlib import Path
test=False

In [6]:
#helper functions for the features.
def _int64_feature(value):
#Wrapper for inserting int64 features into Example proto.
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _bytes_feature(value):
#Wrapper for inserting bytes features into Example proto.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))



def image_example(image_string, label):
#creates the example to write in the TFrecords file
    image_shape = tf.image.decode_image(image_string)
    feature = {
      'height': _int64_feature(image_shape.shape[0]),
      'width': _int64_feature(image_shape.shape[1]),
      'depth': _int64_feature(image_shape.shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
  }
    return tf.train.Example(features=tf.train.Features(feature=feature))


def for_val(folder_path, record_file, val_percentage,numbers=10):
#creates a randomised vector to choose files for validation data
    file_num = 0
    for i in range(numbers):
        folder = i
        data_folder = Path(folder_path + "/%d" %folder)
        directory = os.fsencode(data_folder) 
        for file in os.listdir(directory):
            file_num +=1
    np.random.seed(26)
    to_val = np.random.randint(file_num,size = int(file_num * val_percentage))
    to_val = list(set(to_val))
    to_val = np.sort(to_val)
    lim = len(to_val)-1
    return to_val, lim, file_num

def create_val(folder_path, record_file, numbers=10,val_percentage=0.1):
#creates the validation data tfrecords file
    to_val, lim, _  = for_val(folder_path, record_file, val_percentage, numbers)
    idx = 0
    buffer_size=0
    file_name = 'val_' + record_file
    with tf.io.TFRecordWriter(file_name) as writer_val:
        for i in range(numbers):
            folder = i
            data_folder = Path(folder_path + "/%d" %folder)
            directory = os.fsencode(data_folder) 
            for file in os.listdir(directory):
                filename = data_folder / os.fsdecode(file)
                image_string = open(filename,'rb').read()
                label = folder
                tf_example = image_example(image_string, label)
                if buffer_size == to_val[idx]:
                    writer_val.write(tf_example.SerializeToString())
                    if idx!=lim:
                        idx +=1
                buffer_size +=1

def create_train(folder_path, record_file,numbers = 10, val_percentage=0.1):
#creats the training data tfrecords file
    to_val , lim, _  = for_val(folder_path, record_file, val_percentage, numbers)
    idx = 0
    buffer_size=0
    file_name = 'train_' + record_file                    
    with tf.io.TFRecordWriter(file_name) as writer_train:
        for i in range(numbers):
            folder = i
            data_folder = Path(folder_path + "/%d" %folder)
            directory = os.fsencode(data_folder) 
            for file in os.listdir(directory):
                filename = data_folder / os.fsdecode(file)
                image_string = open(filename,'rb').read()
                label = folder
                tf_example = image_example(image_string, label)
                if buffer_size != to_val[idx]:
                    writer_train.write(tf_example.SerializeToString())
                else:
                    if idx!=lim:
                        idx +=1
                buffer_size +=1
                

if test:
    def create_test(folder_path, record_file,numbers = 10, val_percentage=0.1):
    #creats the training data tfrecords file
       #to_val , lim, _  = for_val(folder_path, record_file, val_percentage, numbers)
        idx = 0
        buffer_size=0
        file_name = 'test_' + record_file                    
        with tf.io.TFRecordWriter(file_name) as writer_test:
            directory = os.fsencode(folder_path) 
            for file in os.listdir(directory):
                filename = data_folder / os.fsdecode(file)

            for i in range(numbers):
                folder = i
                data_folder = Path(folder_path + "/%d" %folder)
                directory = os.fsencode(data_folder) 
                for file in os.listdir(directory):
                    filename = data_folder / os.fsdecode(file)
                    image_string = open(filename,'rb').read()
                    label = folder
                    tf_example = image_example(image_string, label)
                    if buffer_size != to_val[idx]:
                        writer_train.write(tf_example.SerializeToString())
                    else:
                        if idx!=lim:
                            idx +=1
                    buffer_size +=1
    

    
record_file = 'minst.tfrecords'
training_folder_path = 'mnist data/trainingSet/trainingSet'
test_folder_path = 'mnist data/testSet/'
create_val(training_folder_path, record_file)
create_train(training_folder_path, record_file)





In [18]:
def convert_back(data_type,buffer_size,record_file,val_percentage=0.1, channels =1, img_size = (28,28)):
# converts the tfrecords files to images and labels and returns the parsed dataset
    def _parse_image_function(example_proto):
        # Parse the input tf.Example proto using the dictionary above.
        # Create a dictionary describing the features.
        image_feature_description = {
            'height': tf.io.FixedLenFeature([], tf.int64),
            'width': tf.io.FixedLenFeature([], tf.int64),
            'depth': tf.io.FixedLenFeature([], tf.int64),
            'label': tf.io.FixedLenFeature([], tf.int64),
            'image_raw': tf.io.FixedLenFeature([], tf.string),
        }
        image_features = tf.io.parse_single_example(example_proto,image_feature_description)
        image_buffer = image_features['image_raw']
        image = tf.image.decode_jpeg(image_buffer,channels = channels)
        image = tf.image.convert_image_dtype(image,dtype=tf.float32)*(1. / 255)
        image_shape = tf.stack([img_size[0],img_size[1],channels])
        image = tf.reshape(image,image_shape)
        label = tf.cast(image_features['label'],tf.uint8)
        label = tf.squeeze(label)
        return image,label
    
    batch_size = 32
    num_parallel_batches = 2
    if data_type == 'val':
        buffer = int(buffer_size * val_percentage)
    else:
        buffer = buffer_size
    raw_image_dataset = tf.data.TFRecordDataset(data_type+ '_' + record_file)
    raw_image_dataset = raw_image_dataset.shuffle(buffer)
    parsed_image_dataset = raw_image_dataset.map(_parse_image_function, num_parallel_calls = num_parallel_batches)
    parsed_image_dataset = parsed_image_dataset.batch(batch_size)
    parsed_image_dataset = parsed_image_dataset.prefetch(1)
    return parsed_image_dataset


record_file = 'minst.tfrecords'
training_folder_path = 'mnist data/trainingSet/trainingSet'
_, _, buffer_size = for_val(training_folder_path, record_file, val_percentage=0.1,numbers=10)
val_ds = convert_back('val',buffer_size = buffer_size, record_file = record_file)
train_ds = convert_back('train',buffer_size = buffer_size, record_file = record_file)

#sanity check
if not test:
    for image, label in train_ds.take(1):
        print(image.shape, label.shape)

(32, 28, 28, 1) (32,)


In [13]:
filters = 100
kernels = (3,3)
pools = (3,3)
dense1 = 500
dense2 = 250
last_dense = 10
dropout1 = 0.5
dropout2 = 0.4

#creating the model
model = tf.keras.Sequential()
model.add(Conv2D(filters, kernel_size=kernels, padding='same', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=pools, padding='same'))
model.add(Flatten())
model.add(Dense(dense1, activation='relu'))
model.add(Dropout(dropout1))
model.add(Dense(dense2, activation='relu'))
model.add(Dropout(dropout2))
model.add(Dense(last_dense, activation='softmax'))

In [14]:
model.compile(optimizer='adam', metrics=['sparse_categorical_accuracy'], loss='sparse_categorical_crossentropy')

In [19]:
model.fit(train_ds, epochs=10, validation_data = val_ds)

Epoch 1/10
1188/1188 [==============================] - 7s 6ms/step - loss: 0.0967 - sparse_categorical_accuracy: 0.9699 - val_loss: 0.0654 - val_sparse_categorical_accuracy: 0.9818
Epoch 2/10
1188/1188 [==============================] - 5s 4ms/step - loss: 0.0895 - sparse_categorical_accuracy: 0.9717 - val_loss: 0.0715 - val_sparse_categorical_accuracy: 0.9776
Epoch 3/10
1188/1188 [==============================] - 5s 4ms/step - loss: 0.0877 - sparse_categorical_accuracy: 0.9715 - val_loss: 0.0660 - val_sparse_categorical_accuracy: 0.9818
Epoch 4/10
1188/1188 [==============================] - 5s 4ms/step - loss: 0.0841 - sparse_categorical_accuracy: 0.9737 - val_loss: 0.0605 - val_sparse_categorical_accuracy: 0.9830
Epoch 5/10
1188/1188 [==============================] - 5s 4ms/step - loss: 0.0823 - sparse_categorical_accuracy: 0.9742 - val_loss: 0.0580 - val_sparse_categorical_accuracy: 0.9825
Epoch 6/10
1188/1188 [==============================] - 5s 4ms/step - loss: 0.0728 - spars